# Scraping Toots about Switzerland's Migros on Mastodon Social 

This notebook is a prototype on how to scrape messages - so called toots - from Mastodon with Python and Pandas.

Mastodon is like Twitter, a free online service which allows you to send messages to thousands/ millions of people - especially your followers. What is the difference - Masatodon is not a croporate, imagine it as thousands of small Twitters, so called instances. Everyone can build an instance and host it. Some of these instances are public, but for most one has to demand access. We are going to scrape data from an open instance - the Social channel.

When you are scraping from Mastodon consider these two things:

1: Strict research rules are in place - whatever you do, do it in line with the Mastodon community: studies have been withdrawn because of privacy violations (Roel Roscam Abbing, Robert W. Gehl, 2024). This is especially true for the closed instances.

2: Lots of instances have a public facing REST API for allowing users to interact with their services using third-party software. Which makes it very easy to scrape toots for a data project.

So, it is easy to scrape data from Mastodon, but be ethical!

Migros is Switzerland's largest retail company, its largest supermarket chain and largest employer. It is also one of the forty largest retailers in the world. It is structured in the form of a cooperative federation (the Federation of Migros Cooperatives), with more than two million members.

Let's see what Mastodon "toots" about Migros!

FYI: you can simply adapt this to your hashtag of choice, try it out!

## Load libraries and modules

In [1]:
import json
import requests
import pandas as pd

In [2]:
from bs4 import BeautifulSoup

## Preparation - Defining Parameters, Flags, and a Timeframe

The hashtag you're interested in (in this case, 'migros').
The API endpoint that returns posts with the given hashtag from Mastodon. We are scraping from Mastodon's social channel.

In [3]:
hashtag = 'ikea'
URL = f'https://mastodon.social/api/v1/timelines/tag/{hashtag}'

I am  interested in a certain timeframe, which I am going to define in the since variable. 
This has to be set as a timestamp.

In [16]:
since = pd.Timestamp('2024-01-01 00:00:00', tz='UTC')

Obviously, I will work with a loop to gather the toots. is_end is a flag that will help terminate the loop once the condition is met (when no more recent posts are available).

In [17]:
is_end = False

I am going to work with Pythons requests library. The .get function requires an URL - which I already defined. Additional parameters are possible. I am going to work with params, which should be defined as dictionary. First key-value pair is 'limit':40. The API request will ask for a maximum of 40 posts in each request, which is due to the fact that the maximum value of toots that can be pulled at once is 40. Which also means, that the toots coem in as chunks.

In [18]:
params = {
    'limit': 40
}

## Browsing and scraping

The following code now actually fetches recent posts with a specific hashtag from Mastodon and stores them in a Pandas DataFrame. See below for step by step description.

In [19]:
results = []
chunk_no = 1

while True:

    try:
        response = requests.get(URL, params=params)
        print("STATUS OF YOUR SCRAPING")
        print("OK - scraping of chunk no {} worked.".format(chunk_no))
        print("--------------------------------")
        chunk_no += 1
        response.close()
    except:
        print("An error occured." 
              "The http status code is {}".format(response.status_code))
    
    toots = json.loads(response.text)

    if len(toots) == 0:
        print("There were no toots returned. " 
              "Check for spelling or use another hashtag")
        break
    
    for toot in toots:
        timestamp = pd.Timestamp(toot['created_at'], tz='utc')
        if timestamp <= since:
            is_end = True
            break
            
        results.append(toot)
    
    if is_end:
        break
    
    max_id = toots[-1]['id']
    params['max_id'] = max_id

    
df = pd.DataFrame(results)

STATUS OF YOUR SCRAPING
OK - scraping of chunk no 1 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 2 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 3 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 4 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 5 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 6 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 7 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 8 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 9 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 10 worked.
--------------------------------
STATUS OF YOUR SCRAPING
OK - scraping of chunk no 11 worked.
--------

Let's go through the code step by step:

results - An empty list that will store the fetched posts (toots).

chunk_no - a little extra for the loop to count the chunk and follow the process

while True: - A while True loop is initiated to continuously send requests to the API. 

try - except with:

    -- response = requests.get(URL, params=params) - sends a GET request to the Mastodon API.
    -- print statement which lets you follow the status, adding 1 to chunk_no
    -- except statement when something goes wrong - probably the http status code is not 200

toots = json.loads(response.text) - converts the JSON response into a Python object (list of dictionaries). Only the .text attribute is stored.

if len(toots) == 0: break - if there are no toots returned in the response, the loop terminates by using break. A statement is printed.

for loop: The script loops over each toot in the response. For each toot: 

    -- timestamp: The creation time of the toot is converted into a pandas.Timestamp in UTC.
    -- If the toot's timestamp is older than or equal to the since timestamp, the loop breaks, and the fetching ends (is_end  is set to True).
    -- If the toot is within the timeframe, it is appended to the results list.

max_id = toots[-1]['id'] and params['max_id'] = max_id - The last toot's ID (max_id) is stored. And we add max_id to the params dictionary. This way, I ensure that when the next iteration in the loop starts, I will get the next set of toots and not the same again. Saying: The max_id parameter tells the API to return posts older than the specified ID.

if is_end: break - If is_end is set to True (i.e., a post older than the date set in since), the loop terminates.

## Let's have a look at results and df

In [20]:
df.shape

(1010, 24)

As one can see, there are 24 columns - so there is a lot of variables - information stored. Let's have a look at these. 

In [21]:
df.columns

Index(['id', 'created_at', 'in_reply_to_id', 'in_reply_to_account_id',
       'sensitive', 'spoiler_text', 'visibility', 'language', 'uri', 'url',
       'replies_count', 'reblogs_count', 'favourites_count', 'edited_at',
       'content', 'reblog', 'account', 'media_attachments', 'mentions', 'tags',
       'emojis', 'card', 'poll', 'application'],
      dtype='object')

I am only interested in id, created_at, content, account, tags and emojis. Here comes what needs to be considered:

The column account stores values in a dictionary. All kind of info given by the user is stored (websites, note, avatar, emojis, etc.). I am only interested in the account id, which I will use later for some tiny cleaning up. If one wants to use any of the other info stored in account, please make sure to check what can be used and what not.

content is stored as HTML code.

The column tag is a list of dictionaries. I am interested in those, which is why I am using the explode() module on that column.


## Cleaning up the data - respectively the DataFrame

**First, let's extract the account id and build a small subset with only the columns of interest**

In [22]:
df['account_id'] = df['account'].apply(lambda x: x['id'])
df_small = df[["id", "created_at", "language", "account_id", "content", "tags", "emojis"]]
df_small.head()

,id,created_at,language,account_id,content,tags,emojis
0,113211243233112610,2024-09-27T19:47:58.000Z,de,703958,<p>Blåhaj für 13€?<br>Blåhaj für 13€!</p><p>Un...,"[{'name': 'ikea', 'url': 'https://mastodon.soc...",[]
1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"[{'name': 'hygiene', 'url': 'https://mastodon....",[]
2,113210405795153302,2024-09-27T16:15:01.721Z,en,112981119972404972,<p>AI is transforming business and society as ...,"[{'name': 'ai', 'url': 'https://mastodon.socia...",[]
3,113209999111602024,2024-09-27T14:31:33.000Z,de,110723712695064921,<p>Menschen haben ein Babyfläschen im Aufwärme...,"[{'name': 'ikea', 'url': 'https://mastodon.soc...",[]
4,113206733632078239,2024-09-27T00:41:08.000Z,nl,110360759253638280,<p>Verkoopmedewerker Keukens (fulltime) – Ikea...,"[{'name': 'vacature', 'url': 'https://mastodon...",[]


In [23]:
df_small.dtypes

id            object
created_at    object
language      object
account_id    object
content       object
tags          object
emojis        object
dtype: object


**You might want to skip the following step, if you're not using the migros hashtag**

It is always a good idea to browse the posts which have the researched hashtags on the Mastadon website - in this case Mastadon Social, to get a first impression.

I realized, that there is an account which posts about all things business, entrepreneurship in Switzerland but always in German AND in French. It would obviously be the same content, so I decided to drop the account posting the message in French and to keep the one posting the content in German.

In [33]:
id_to_drop = ['109471764792431240']
df_clean = df_small[df_small.account_id.isin(id_to_drop) == False]
df_clean.head()

,id,created_at,language,account_id,content,tags,emojis
0,113204241376169101,2024-09-26T14:07:19.000Z,de,109669545373361861,<p>Migros rechnet mit minimal höherem Betriebs...,"[{'name': 'migros', 'url': 'https://mastodon.s...",[]
2,113179882647913861,2024-09-22T06:52:31.000Z,de,109415182274164306,"<p><a href=""https://smnn.ch/tags/Qualcomm"" cla...","[{'name': 'qualcomm', 'url': 'https://mastodon...",[]
3,113160216482271211,2024-09-18T19:30:43.000Z,en,109248094876977535,"<p>…wann explodieren die <a href=""https://mast...","[{'name': 'grapefruits', 'url': 'https://masto...",[]
4,113153493068774612,2024-09-17T15:01:22.000Z,de,109669545373361861,<p>Migros schliesst Webshops von Tochterfirmen...,"[{'name': 'migros', 'url': 'https://mastodon.s...",[]
5,113117546765206911,2024-09-11T06:39:45.000Z,de,109669545373361861,<p>Migros lanciert mit Migros Bank kostenloses...,"[{'name': 'migrosbank', 'url': 'https://mastod...",[]


**Using explode on the tags column**

As mentioned before, the tags column is a list of dictionaries, which is why I am going to use the explode() function. Every dictionary contains information about one hashtag used in the post. So number of hashtags = number of dictionaries. Included is the name and hashtag url.

What happens, when using explode: every dictionary (so every hashtag) get's it's own row, while the values of the other columns are copied and stay the same. So every tag will end on it's own line, which is why I will keep the index column, there I can easily follow which rows belonged to the same post.

In [30]:
df_flat = df_small.explode("tags").reset_index()
df_flat.head(20)

,index,id,created_at,language,account_id,content,tags,emojis
0,0,113211243233112610,2024-09-27T19:47:58.000Z,de,703958,<p>Blåhaj für 13€?<br>Blåhaj für 13€!</p><p>Un...,"{'name': 'ikea', 'url': 'https://mastodon.soci...",[]
1,0,113211243233112610,2024-09-27T19:47:58.000Z,de,703958,<p>Blåhaj für 13€?<br>Blåhaj für 13€!</p><p>Un...,"{'name': 'blahaj', 'url': 'https://mastodon.so...",[]
2,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"{'name': 'hygiene', 'url': 'https://mastodon.s...",[]
3,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"{'name': 'ratgeber', 'url': 'https://mastodon....",[]
4,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"{'name': 'ikea', 'url': 'https://mastodon.soci...",[]
5,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"{'name': 'verbraucher', 'url': 'https://mastod...",[]
6,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"{'name': 'schlafstorungen', 'url': 'https://ma...",[]
7,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"{'name': 'schlaf', 'url': 'https://mastodon.so...",[]
8,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"{'name': 'tests', 'url': 'https://mastodon.soc...",[]
9,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,"{'name': 'okotest', 'url': 'https://mastodon.s...",[]


**Cleaning a little more the tags column**

The tags column includes the name of the tag as well as the url - which one could click to follow the hashtag... But I only need the name, which is also easier to grasp at first sight. The following code will create a new column tag_name, I will drop the original tags column later on, when I create a new DataFrame.

In [25]:
df_flat['tag_name'] = df_flat['tags'].apply(lambda y: y['name'])

**Extracting the text from the content column**

As pointed out earlier, the content column is in HTML code. Defining a function which utilizes BeautifulSoup to parse the code. Again creating a new column extracted_content which stores the result. Going to drop the original content column in the next step - and create a new DataFrame.

In [26]:
def extract_text_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text()

df_flat["extracted_content"] = df_flat["content"].apply(extract_text_from_html)

**Narrowing down the DataFrame again**

Dropping the original tags and content column.

In [27]:
df_flat_small = df_flat[[
    "index", 
    "id", 
    "created_at", 
    "language", 
    "account_id", 
    "extracted_content", 
    "tag_name", 
    "emojis"]]

In [28]:
df_flat_small

,index,id,created_at,language,account_id,extracted_content,tag_name,emojis
0,0,113211243233112610,2024-09-27T19:47:58.000Z,de,703958,Blåhaj für 13€?Blåhaj für 13€!Und gratis heiße...,ikea,[]
1,0,113211243233112610,2024-09-27T19:47:58.000Z,de,703958,Blåhaj für 13€?Blåhaj für 13€!Und gratis heiße...,blahaj,[]
2,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,hygiene,[]
3,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,ratgeber,[]
4,1,113210755701475817,2024-09-27T17:22:14.000Z,de,111200287121234070,Die Wahl des richtigen Kissens zum Schlafen wi...,ikea,[]
...,...,...,...,...,...,...,...,...
3931,1007,111687735502160379,2024-01-02T18:19:50.000Z,en,110253195097833332,Man confesses to reading the instructions#IKEA...,satire,[]
3932,1008,111686272684337303,2024-01-02T12:07:54.000Z,nl,109273418053113580,Naar #Ikea geweest voor #kunstplanten. We hebb...,kunstplanten,[]
3933,1008,111686272684337303,2024-01-02T12:07:54.000Z,nl,109273418053113580,Naar #Ikea geweest voor #kunstplanten. We hebb...,ikea,[]
3934,1009,111682466230388020,2024-01-01T19:59:52.000Z,en,109943090412221144,#IKEA ist doch nicht mehr ganz dicht?! Oh mome...,ikea,[]


## Having a closer look at the hashtags

I am going to use value_counts() in the tag_name column to count the appearance of each hashtag. Creating a new DataFrame with just the hashtags and their individual count.

In [29]:
tags_count = df_flat_small["tag_name"].value_counts().reset_index()
tags_count.head(25)

,tag_name,count
0,ikea,1010
1,blahaj,26
2,zpravicky,19
3,arte,18
4,diy,17
5,furniture,16
6,news,16
7,smarthome,16
8,tradfri,15
9,homeassistant,15


**Result**

So, there is not much intersting to see, also: we now have a time frame from the 1st of January 2020 and we have 210 posts, which ist not that much. So not so much posted about Migros on Mastodon. Nevertheless - and the technique remains the same - we can now have a lokk at certain hashtags.

Of course, there is Migros - as we searched for that one. Then Coop - another Swiss supermarket (in fact, as well a big cooperative) - is mentioned quite often together with Migros. We have companies mentioned which belong to the cooperate: Denner, Micarna, Migrolino.

I am interested in the posts which mention nachhaltig (sustainable), nachhaltigkeit (sustainability) and greenwashing. Sustainability is an important topic. Let's have a look at the content of these posts and see what people think.

In [52]:
df_sustainable = df_flat_small[(df_flat_small["tag_name"] == "nachhaltig") | (df_flat_small["tag_name"] == "nachhaltigkeit") | (df_flat_small["tag_name"] == "greenwashing")]

In [53]:
sustainable = df_new["extracted_content"].to_list()
sustainable

['Finde den Fehler!5 Jahre alte #Migros vs. neue #Rewe Tasche.100% vs. 36% :mastodon:  #nachhaltig #recycling #Plastik #pet #greenwashing @rewe',
 'Finde den Fehler!5 Jahre alte #Migros vs. neue #Rewe Tasche.100% vs. 36% :mastodon:  #nachhaltig #recycling #Plastik #pet #greenwashing @rewe',
 'Was bedeutet #nachhaltig, #klimaneutral oder #natürlich? @ErichBuergler berichtet in der @sonntagszeitung über unsere Kritik an der #Werbung von #Migros und #Coop(Abo-Artikel).https://www.tagesanzeiger.ch/greenpeace-wirft-migros-und-coop-greenwashing-vor-243204184726',
 '«\xa0Die grünen Werbeslogans von #migros und #coop  sind laut der Umweltschutzorganisation Greenpeace undurchsichtig. Auch die Wissenschaft kritisiert das Marketing von Schweizer Unternehmen.#lebensmittel #detailhandel #nachhaltig #nachhaltigkeit https://lnkd.in/eASV8eM6\xa0»— Retweet https://twitter.com/ErichBuergler/status/1647532695696572417',
 '«\xa0Die grünen Werbeslogans von #migros und #coop  sind laut der Umweltschutzorgan

In [54]:
df_schlachthof = df_flat_small[df_flat_small["tag_name"] == "schlachthof"]
df_schlachthof

,index,id,created_at,language,account_id,extracted_content,tag_name,emojis
491,169,110615042718162477,2023-06-27T07:40:02.000Z,en,386737,#Schlachthof #Migros: Die Gemeinde St-Aubin ha...,schlachthof,[]
570,196,110144756385512536,2023-04-05T06:20:02.000Z,en,386737,#Migros plant mit ihrer Tochtergesellschaft #M...,schlachthof,[]


In [55]:
schlachthof = df_schlachthof["extracted_content"].to_list()
schlachthof

['#Schlachthof #Migros: Die Gemeinde St-Aubin hat unsere Einsprache abgelehnt. Wir legen nun beim Kanton Freiburg Rekurs ein. Gemeinde und Kanton sollten der Umwelt Priorität einräumen können, nicht den wirtschaftlichen Interessen der Migros-Gruppe.https://act.gp/442XspQ',
 '#Migros plant mit ihrer Tochtergesellschaft #Micarna, einen gigantischen #Schlachthof im Herzen der Westschweiz zu bauen. 🐔 40 Millionen Hühner sollen da jährlich getötet werden. 🐔Wir stellen uns gegen dieses umwelt- und klimaschädliche Projekt. 👇https://act.gp/3zu7qCZ']